# Ejercicios Dataframes 2
Realiza los siguientes ejercicios. Soluciona cada uno de ellos empleando la DataFrame API y Spark SQL
1. Inicializa la variable spark (0)

In [1]:
#DataFrame API
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
print (spark.version)
#SQL



:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hadoop/.ivy2/cache
The jars for the packages stored in: /home/hadoop/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-949f2da9-2fd7-45a0-b8b3-c6f1545adf26;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.7 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.fi

3.5.7


2. Crea un dataframe llamado mi_df con los datos del archivo *data/retail-data/all/online-retail-dataset.csv* (0)

In [2]:
#DataFrame API
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType
# Definir el esquema
schema = StructType([
    StructField("InvoiceNo", IntegerType(), True),
    StructField("StockCode", StringType(), True),
    StructField("Description", StringType(), True),
    StructField("Quantity", IntegerType(), True),
    StructField("InvoiceDate", TimestampType(), True),
    StructField("UnitPrice", DoubleType(), True),
    StructField("CustomerID", IntegerType(), True),
    StructField("Country", StringType(), True)
])

mi_df = spark.read.csv("hdfs:///data/retail-data/all/online-retail-dataset.csv", header=True, schema=schema)
mi_df.show(5)
mi_df.printSchema()

# Leer el archivo CSV con el esquema especificado

#SQL
mi_df.createOrReplaceTempView("retail")



+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|       NULL|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|       NULL|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|       NULL|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|       NULL|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|       NULL|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
only showing top 5 rows

root
 |-- InvoiceNo: integer (nullable = true)
 |-- StockCode: string (null

3. Cuenta el número de celdas totales

In [ ]:
from pyspark.sql.functions import count
#DataFrame API
mi_df.select(count("*")).show()

#SQL
spark.sql("select count(*) from retail").show()


4. Cuenta el número de "invoiceNo" distintos

In [ ]:
#DataFrame API
print(mi_df.select("InvoiceNo").distinct().count())

#SQL
spark.sql("select count(distinct InvoiceNo) from retail").show()


5. Obtén el número de factura más bajo y el más alto

In [6]:
#DataFrame API
mi_df.selectExpr("max(InvoiceNo) as max_factura", "min(InvoiceNo) as min_factura").show()

#SQL
spark.sql("select max(InvoiceNo), min(InvoiceNo) from retail").show()


+-----------+-----------+
|max_factura|min_factura|
+-----------+-----------+
|     581587|     536365|
+-----------+-----------+



[Stage 10:==============================================>          (9 + 1) / 11]

+--------------+--------------+
|max(InvoiceNo)|min(InvoiceNo)|
+--------------+--------------+
|        581587|        536365|
+--------------+--------------+



6. Obtén la suma de todos los importes unitarios de los productos vendidos en el Reino Unido

In [18]:
from pyspark.sql.functions import col,sum
#DataFrame API
mi_df.filter(col("Country")=="United Kingdom").select(sum("UnitPrice").alias("suma")).show()
spark.sql("select sum(UnitPrice) as suma from retail where Country = 'United Kingdom'").show()


+-----------------+
|             suma|
+-----------------+
|2245715.474000272|
+-----------------+



[Stage 54:==============================================>          (9 + 1) / 11]

+-----------------+
|             suma|
+-----------------+
|2245715.474000272|
+-----------------+



7. Obtén la media de todos los importes unitarios de los productos vendidos en el Reino Unido

In [21]:
from pyspark.sql.functions import avg
#DataFrame API
mi_df.filter(col("Country")=="United Kingdom").select(avg("UnitPrice").alias("media")).show()

#SQL

spark.sql("select avg(UnitPrice) as media from retail where Country = 'United Kingdom'").show()

+-----------------+
|            media|
+-----------------+
|4.532422174143497|
+-----------------+



[Stage 66:=========================================>               (8 + 1) / 11]

+-----------------+
|            media|
+-----------------+
|4.532422174143497|
+-----------------+



8. Obtén la el número total de productos vendidos (quantity) agrupado por países (muestra los 10 primeros)

In [4]:
#DataFrame API
mi_df.groupBy("Country").sum("quantity").show()

#SQL
spark.sql("SELECT Country, sum(Quantity) FROM retail GROUP BY Country").show()


+---------------+-------------+
|        Country|sum(quantity)|
+---------------+-------------+
|      Australia|        83653|
|        Denmark|         8188|
|    Switzerland|        30325|
|        Belgium|        23152|
|           EIRE|       142637|
|    Netherlands|       200128|
|         Israel|         4353|
|        Austria|         4827|
|          Japan|        25218|
|Channel Islands|         9479|
|         Cyprus|         6317|
|        Finland|        10666|
|        Bahrain|          260|
|          Spain|        26824|
|         Poland|         3653|
|        Iceland|         2458|
|         Norway|        19247|
|        Germany|       117448|
|         Sweden|        35637|
| United Kingdom|      4263829|
+---------------+-------------+
only showing top 20 rows

+---------------+-------------+
|        Country|sum(Quantity)|
+---------------+-------------+
|      Australia|        83653|
|        Denmark|         8188|
|    Switzerland|        30325|
|        Belgi

9. Obtén la media de los precios unitarios de los productos vendidos agrupada por países

In [6]:
#DataFrame API
#DataFrame API
print("API Dataframes:")
mi_df.groupBy("Country").avg("UnitPrice").show()

#SQL
print("SQL:")
spark.sql("SELECT Country, avg(UnitPrice) FROM retail GROUP BY Country").show()


API Dataframes:
+---------------+------------------+
|        Country|    avg(UnitPrice)|
+---------------+------------------+
|      Australia|3.2206115965051607|
|        Denmark|  3.25694087403599|
|    Switzerland|  3.40344155844156|
|        Belgium|3.6443354277428726|
|           EIRE| 5.911077354807234|
|    Netherlands|2.7383171657528482|
|         Israel| 3.633131313131314|
|        Austria| 4.243192019950122|
|          Japan|2.2761452513966476|
|Channel Islands| 4.932124010554089|
|         Cyprus| 6.302363344051452|
|        Finland| 5.448705035971222|
|        Bahrain| 4.556315789473685|
|          Spain| 4.987544413738653|
|         Poland| 4.170879765395893|
|        Iceland|2.6440109890109897|
|         Norway|6.0120257826887675|
|        Germany| 3.966929963138483|
|         Sweden| 3.910887445887446|
| United Kingdom| 4.532422174143497|
+---------------+------------------+
only showing top 20 rows

SQL:
+---------------+------------------+
|        Country|    avg(Uni

10. Obtén el importe total (quantity * unit price) agrupado por número de factura (invoiceNo)

In [7]:
from pyspark.sql.functions import expr
#DataFrame API
print("API Dataframes:")
mi_df.withColumn("total", expr("Quantity * UnitPrice")).groupBy("InvoiceNo").sum("total").show()
#SQL
print("SQL:")
spark.sql("SELECT InvoiceNo, sum (Quantity * UnitPrice) FROM retail GROUP BY InvoiceNo").show()


API Dataframes:


+---------+------------------+
|InvoiceNo|        sum(total)|
+---------+------------------+
|   536533|156.09999999999997|
|   537049|            277.65|
|   537057|274.66999999999996|
|   537155| 75.64000000000001|
|   537256|2263.7999999999993|
|   537350|            419.06|
|   537419|510.94999999999993|
|   537800|             69.79|
|   537817|             168.6|
|   537842|159.51000000000002|
|   538077| 686.1499999999999|
|   538096|3.7199999999999998|
|   538133|               0.0|
|   538868|            824.02|
|   539064|              58.7|
|   539246|104.48000000000002|
|   539561| 339.7200000000001|
|   539637|112.68000000000002|
|   539660|               9.2|
|   539958| 5262.059999999995|
+---------+------------------+
only showing top 20 rows

SQL:
+---------+---------------------------+
|InvoiceNo|sum((Quantity * UnitPrice))|
+---------+---------------------------+
|   536533|         156.09999999999997|
|   537049|                     277.65|
|   537057|         274.6

11. Crea un DataFrame para cada archivo de notas (notas_fisica, notas_ingles y notas_matemáticas). Realiza un Join que genere un DataFrame mostrando las tres notas para cada alumno. (sólo mostrar los alumnos que tengan nota en las 3 asignaturas)

In [10]:
#DataFrame API

print ("API DataFrames: ")
notas_fisica = spark.read.option("header",False).option("inferSchema",True).csv("/data/notas/notas_fisica.txt").toDF("alumno","nota_fisica")
notas_ingles = spark.read.option("header",False).option("inferSchema",True).csv("/data/notas/notas_ingles.txt").toDF("alumno","nota_ingles")
notas_matematicas = spark.read.option("header",False).option("inferSchema",True).csv("/data/notas/notas_mates.txt").toDF("alumno","nota_matematicas")


notas_join = notas_fisica.join(notas_matematicas, "alumno", "inner").join(notas_ingles, "alumno", "inner")
notas_join.show()
#.join(notas_ingles)
#notas_join.show()

#SQL
print ("SQL: ")
notas_fisica.createOrReplaceTempView("fisica")
notas_matematicas.createOrReplaceTempView("matematicas")
notas_ingles.createOrReplaceTempView("ingles")
spark.sql("SELECT fisica.alumno, nota_fisica, nota_matematicas, nota_ingles FROM fisica, matematicas, ingles WHERE fisica.alumno = matematicas.alumno and matematicas.alumno = ingles.alumno").show()







API DataFrames: 
+---------+-----------+----------------+-----------+
|   alumno|nota_fisica|nota_matematicas|nota_ingles|
+---------+-----------+----------------+-----------+
|    Angel|          9|             6.0|          4|
|    Maria|          3|             2.0|          6|
|    Ramon|          7|             4.5|          8|
|    Jorge|          5|            10.0|          5|
|   Susana|          9|             9.0|          2|
|   Anabel|          2|             8.0|          7|
|    Rocio|          5|             6.0|          4|
|   Carlos|          4|             4.0|          8|
|    Rocio|          7|             6.0|          4|
|   Triana|          3|             3.0|          4|
|   Andres|          4|             4.0|          6|
| Fernando|          9|             5.0|          7|
| Leonardo|          6|             1.0|          4|
|    Oscar|          5|             7.0|          3|
|   Isabel|          8|             8.0|          7|
|Jose Juan|          3|      

12. Crea un DataFrame para cada archivo de notas (notas_fisica, notas_ingles y notas_matemáticas). Realiza un Join que genere un DataFrame mostrando las tres notas para cada alumno. Si a un alumno le falta alguna de las notas aparecerá el valor NULL

In [11]:
#DataFrame API
print("API DataFrames: ")
notas_outer_join = notas_fisica.join(notas_matematicas, "alumno", "full_outer").join(notas_ingles, "alumno", "full_outer")
notas_outer_join.show()

#SQL
print ("SQL:")
spark.sql("SELECT fisica.alumno, nota_fisica, nota_matematicas, nota_ingles FROM fisica FULL OUTER JOIN matematicas ON fisica.alumno = matematicas.alumno FULL OUTER JOIN ingles ON coalesce(fisica.alumno, matematicas.alumno) = ingles.alumno").show()



API DataFrames: 
+---------+-----------+----------------+-----------+
|   alumno|nota_fisica|nota_matematicas|nota_ingles|
+---------+-----------+----------------+-----------+
|Alejandro|          3|             5.0|          7|
|   Anabel|          2|             8.0|          7|
|   Andres|          4|             4.0|          6|
|    Angel|          9|             6.0|          4|
|   Carlos|          4|             4.0|          8|
| Fernando|          9|             5.0|          7|
|   Isabel|          8|             8.0|          7|
|    Jorge|          5|            10.0|          5|
|Jose Juan|          3|             5.0|          3|
| Leonardo|          6|             1.0|          4|
|    Maria|          3|             2.0|          6|
|  Nicolas|          7|             2.0|          5|
|    Oscar|          5|             7.0|          3|
|    Pedro|          2|             5.0|       NULL|
|    Ramon|          7|             4.5|          8|
|    Rocio|          5|      